# Import Resources

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import numpy as np
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import pymysql
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


# **DATA BASE**

In [2]:
## VARIABLE DEFINITIONS

ssh_ip_address = ''
ssh_p_username = ''
ssh_p_password = ""
ssh_port = 22
ssh_remote_bind_address = 3306
db_server_ip = ''
db_server_port = 3306
db_user = ''
db_user_password = ''
db_name = ''

## SSH CONNECTION

tunnel = SSHTunnelForwarder((ssh_ip_address, 22), ssh_password=ssh_p_password, ssh_username=ssh_p_username,
                            remote_bind_address=('127.0.0.1', 3306))
tunnel.start()
# Establish an SSH tunnel to the remote server using the specified IP address, port, username, and password
# The tunnel will bind to the local address '127.0.0.1' and port 3306

# MYSQL CONNECTION
db = pymysql.connect(host='localhost', port=tunnel.local_bind_port, user=ssh_p_username,
                          passwd=db_user_password, db=db_name)
# Establish a connection to the MySQL server using the local bind port of the SSH tunnel, username, password, and database name


# Create a cursor object to execute SQL queries on the database
cur = db.cursor()

In [3]:
# SQL query to select all data from the table
query = "SELECT * FROM TI_PV_main;"
# Execute the query
cur.execute(query)
# Fetch all the rows
rows1 = cur.fetchall()

# SQL query to select all data from the table
query = "SELECT * FROM TI_PV_branch;"
# Execute the query
cur.execute(query)
# Fetch all the rows
rows2 = cur.fetchall()


df1 = pd.DataFrame(rows1)
df2 = pd.DataFrame(rows2)

# Drop two columns from the first DataFrame
columns_to_drop = [0, 1]
df2 = df2.drop(columns=columns_to_drop)

# # Concatenate the DataFrames
df_c = pd.concat([df1, df2], axis=1)
Labels=df_c[df_c.columns[1]].tolist()

X= df_c.drop(df_c.columns[:2], axis=1)

# Modify labels
for i in range(0,len(Labels)):
    if Labels[i] =='No-Anomaly':
        pass
    else:
        Labels[i]='Anomaly'


# Create an instance of the MinMaxScaler
scaler = MinMaxScaler()
# Normalize the data in the DataFrame
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
y=Labels



In [4]:
y=Labels

# Prepared data for training

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=55)

In [6]:
# To select the top 20 most important features using mutual_info_classif:  https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10) 
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.1, random_state=55)

In [6]:
X_train.shape

(18000, 1000)

# Nearest centroid classifier

In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import neighbors
from sklearn import metrics


In [46]:
# baseline
data= datasets.load_digits()
X_train1, X_test1, y_train1,y_test1 = train_test_split(data.data, data.target)
print(X_train1.shape , X_test1.shape)

(1347, 64) (450, 64)


In [47]:
clf= neighbors.NearestCentroid(metric='correlation',shrink_threshold=0.5) #{'minkowski', 'braycurtis', 'hamming', 'russellrao', 'euclidean', 'cosine', 'correlation', 'dice', 'l2', 'cityblock', 'haversine', 'kulsinski', 'sokalmichener', 'sokalsneath', 'jaccard', 'yule', 'manhattan', 'l1', 'rogerstanimoto', 'chebyshev', 'sqeuclidean', 'matching', 'canberra', 'nan_euclidean'}
clf.fit(X_train,y_train)

y_pred= clf.predict(X_test)

acu= metrics.accuracy_score(y_test, y_pred)
print( 'Acurracy:', acu)

cm= metrics.confusion_matrix(y_test, y_pred)
print( 'Confusion Matrix:', acu)



Acurracy: 0.8888888888888888
Confusion Matrix: 0.8888888888888888


# K Neghbors Classifier

In [48]:
clf= neighbors.KNeighborsClassifier(n_neighbors=30 , algorithm="auto") # algorithm= {‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
clf.fit(X_train,y_train)

y_pred= clf.predict(X_test)

acu= metrics.accuracy_score(y_test, y_pred)
print( 'Acurracy:', acu)

cm= metrics.confusion_matrix(y_test, y_pred)
print( 'Confusion Matrix:', acu)


Acurracy: 0.96
Confusion Matrix: 0.96


# K neihbors +  NCA (Neighborhood component analysis)

In [13]:
X_train.shape

(18000, 10)

In [64]:
nca = neighbors.NeighborhoodComponentsAnalysis(n_components=63)
nca.fit(X_train,y_train)
print(nca)

NeighborhoodComponentsAnalysis(n_components=63)


In [65]:
#KNN
clf= neighbors.KNeighborsClassifier()
clf.fit(nca.transform(X_train),y_train)

y_pred= clf.predict(nca.transform(X_test))

acu= metrics.accuracy_score(y_test, y_pred)
print( 'Acurracy:', acu)

cm= metrics.confusion_matrix(y_test, y_pred)
print( 'Confusion Matrix:', acu)

Acurracy: 0.98
Confusion Matrix: 0.98


# Radius Neighbors Classifier

In [66]:
X_train.shape

(1347, 64)

In [72]:
#KNN
clf= neighbors.RadiusNeighborsClassifier(radius=29)
clf.fit(X_train,y_train)

y_pred= clf.predict(X_test)

acu= metrics.accuracy_score(y_test, y_pred)
print( 'Acurracy:', acu)

cm= metrics.confusion_matrix(y_test, y_pred)
print( 'Confusion Matrix:', acu)

Acurracy: 0.9755555555555555
Confusion Matrix: 0.9755555555555555


# Precompute k nearest neighbors

In [37]:
X_train.shape

(18000, 1000)

In [73]:
graph= neighbors.KNeighborsTransformer(mode='distance',n_neighbors=5, algorithm='brute') #mode{‘distance’, ‘connectivity’}, default=’distance’   algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
graph.fit(X_train, y_train)
print(graph)

KNeighborsTransformer(algorithm='brute')


In [74]:
#KNN
clf= neighbors.KNeighborsClassifier(n_neighbors=1, weights= "distance")
clf.fit(graph.transform(X_train),y_train)

y_pred= clf.predict(graph.transform(X_test))

acu= metrics.accuracy_score(y_test, y_pred)
print( 'Acurracy:', acu)

cm= metrics.confusion_matrix(y_test, y_pred)
print( 'Confusion Matrix:', acu)

Acurracy: 0.9733333333333334
Confusion Matrix: 0.9733333333333334
